In [ ]:
import boto3
import requests
import os
import json

with open('2025a1.json', 'r') as file:
    data = json.load(file)

s3 = boto3.client('s3', region_name='us-east-1')  
bucket_name = 'rmit-music-images'  

# Create the bucket if it doesn't exist
try:
    s3.create_bucket(Bucket=bucket_name)
    print(f"Bucket '{bucket_name}' created successfully.")
except Exception as e:
    print(f"Bucket '{bucket_name}' already exists or cannot be created: {e}")


if not os.path.exists('downloaded_images'):
    os.makedirs('downloaded_images')

for song in data['songs']:
    img_url = song['img_url']
    img_name = img_url.split("/")[-1] 

    try:
        response = requests.get(img_url)
        response.raise_for_status() 

        img_path = os.path.join('downloaded_images', img_name)
        with open(img_path, 'wb') as img_file:
            img_file.write(response.content)

        print(f"Downloaded {img_name} successfully.")

        s3.upload_file(
            Filename=img_path,
            Bucket=bucket_name,
            Key=f'images/{img_name}',
        )
        print(f"Uploaded {img_name} to S3 bucket '{bucket_name}' successfully.")

        # Remove the downloaded image after uploading to S3
        os.remove(img_path)

    except requests.RequestException as e:
        print(f"Failed to download {img_name}: {e}")
    except Exception as e:
        print(f"Failed to upload {img_name} to S3: {e}")

os.rmdir('downloaded_images')
